# Finding fixed effects

## The simple way, empirical approach

In [6]:
import spdepy as sp
import numpy as np
data = sp.datasets.get_sinmod_training()
dataV = sp.datasets.get_sinmod_validation()

In [9]:
tmp0 = 1/(data['mu'].mean() - (dataV['dataS'].mean(axis = (0,1)))).var()
print("tau_beta0 = ",tmp0)
tmp1 = 1/((data['mu'].var()/(dataV['dataS'].var(axis = (0,1)))).var()*data['mu'].max()**2)
print("tau_beta1 = ",tmp1)
np.save("../fits/sinmod/sigmas.npy",np.hstack([tmp0,tmp1]))

tau_beta0 =  0.7050724788436276
tau_beta1 =  0.012955065092673372


## Resampling approach

In [ ]:
import spdepy as sp
import numpy as np

data = sp.datasets.get_sinmod_training()
dataV = sp.datasets.get_sinmod_validation()

mod = sp.model(grid = sp.grid(x=data['x'], y=data['y'],extend = 5),
         spde = 'whittle-matern',ha = True, bc = 3, anisotropic = True,parameters= np.load('../fits/whittle_matern_ha_bc3.npy'))

sigmas = np.log(np.array([0.00728252, 0.27113878]))
mod.setModel(mu = data['mu'],sigmas = sigmas,useCov = True)


In [ ]:
# least squares approach ("resampling")
X = mod.grid.S[:,-2:]
S = mod.grid.S[:,:-2]
Q = mod.Q[:-2,:-2]
y = dataV['dataS'][:,0,0]
betas = np.zeros((72*10,2))
for i in range(72):
    for j in range(10):
        y = dataV['dataS'][:,j,i]
        betas[i*10+j,:] = np.linalg.inv(X.T@S@Q@S.T@X.A)@X.T@S@Q@S.T@y

## Likelihood based approach

In [ ]:
# likelihood based approach
S = mod.grid.getS()
def func(sigmas):
    res = np.zeros((dataV['dataS'].shape[2],11))
    dres = np.zeros((dataV['dataS'].shape[2],11,2))
    for j in range(dataV['dataS'].shape[2]):
        for i in range(10):
            mod.setModel(mu = data['mu'],sigmas = sigmas,useCov = True)
            mod.update(y = dataV['dataS'][dataV['idxS'][:,0][dataV['tS'][:,0]==i],i,j],idx = dataV['idxS'][:,0][dataV['tS'][:,0]==i],grad = True)
            tmp1 = dataV['dataS'][:,i:,0] - (S@mod.mu)[:,np.newaxis]
            tmp2 = np.stack([S.T@tmp1[:,k] for k in range(10-i)],axis = 1)
            res[j,i] = (tmp2.T@mod.Q@tmp2).diagonal().mean()
            dres[j,i,0] = np.mean(-S.T@(S@mod.dmu[:,0])@mod.Q@tmp2 + (tmp2.T@mod.dQ1@tmp2).diagonal()- (tmp2.T@mod.Q@(S.T@S@mod.dmu[:,0])))
            dres[j,i,1] = np.mean(-S.T@(S@mod.dmu[:,1])@mod.Q@tmp2 + (tmp2.T@mod.dQ2@tmp2).diagonal()- (tmp2.T@mod.Q@(S.T@S@mod.dmu[:,1])))
        mod.setModel(mu = data['mu'],sigmas = sigmas,useCov = True)
        tmp1 = dataV['dataS'][:,:,0] - (S@mod.mu)[:,np.newaxis]
        tmp2 = np.stack([S.T@tmp1[:,k] for k in range(10)],axis = 1)
        res[j,i] = (tmp2.T@mod.Q@tmp2).diagonal().mean()
        dres[j,i,0] = np.mean(-S.T@(S@mod.dmu[:,0])@mod.Q@tmp2 + (tmp2.T@mod.dQ1@tmp2).diagonal()- (tmp2.T@mod.Q@(S.T@S@mod.dmu[:,0])))
        dres[j,i,1] = np.mean(-S.T@(S@mod.dmu[:,1])@mod.Q@tmp2 + (tmp2.T@mod.dQ2@tmp2).diagonal()- (tmp2.T@mod.Q@(S.T@S@mod.dmu[:,1])))
    res = np.mean(res)
    dres = np.mean(dres,axis = (0,1))
    print("res = ",res," , grad = ", dres, " , sigmas = ",np.exp(sigmas))
    return(res,-dres)

sigmas = np.array([-2.69984669,  0.10283444])

opt = sp.optim(fun = func)
opt.fit(x0 = sigmas, lr = np.linspace(0.1,0.001,30),end = "../fits/sinmod/whittle_matern_ha_bc3_sigmas2")